In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [47]:
speeches = pd.read_parquet('src/Data/ecb-speeches.parquet')
releases = pd.read_csv("src/Data/ecb_releases_302.csv")
announcements = pd.read_csv("src/Data/policy_announcements.csv")

In [48]:
pd.set_option('display.max_colwidth', 50)

all_data = pd.concat([speeches, releases, announcements], ignore_index=True)
all_data

,date,content
0,1998-07-17,Mr. Duisenberg reports on the outcome of the s...
1,1998-09-18,Mr. Duisenberg’s opening statement at the meet...
2,1998-10-12,Mr. Duisenberg speaks on changes in European f...
3,1998-10-12,Mr. Duisenberg's statement to the European Par...
4,1998-10-22,Mr. Duisenberg’s opening statement at the pres...
...,...,...
2846,8 May 2014,"At today’s meeting, which was held in Brussels..."
2847,3 April 2014,At today’s meeting the Governing Council of th...
2848,6 March 2014,At today’s meeting the Governing Council of th...
2849,6 February 2014,At today’s meeting the Governing Council of th...


In [49]:
import string
import nltk
from nltk.corpus import stopwords

# Download stopwords if you haven't already
nltk.download('stopwords')

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Replace newline characters with space
    text = text.replace('\n', ' ')

    # Replace hyphenated words with underscore
    text = ' '.join([word.replace('-', '_') if '-' in word else word for word in text.split()])
    
    # Replace punctuation with space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Strip extra whitespaces
    text = ' '.join(text.split())
    
    return text

all_data['content_clean'] = all_data['content'].apply(clean_text)
all_data


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ozodbek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,date,content,content_clean
0,1998-07-17,Mr. Duisenberg reports on the outcome of the s...,mr duisenberg reports outcome second meeting g...
1,1998-09-18,Mr. Duisenberg’s opening statement at the meet...,mr duisenberg’s opening statement meeting gove...
2,1998-10-12,Mr. Duisenberg speaks on changes in European f...,mr duisenberg speaks changes european financia...
3,1998-10-12,Mr. Duisenberg's statement to the European Par...,mr duisenberg statement european parliament’s ...
4,1998-10-22,Mr. Duisenberg’s opening statement at the pres...,mr duisenberg’s opening statement press confer...
...,...,...,...
2846,8 May 2014,"At today’s meeting, which was held in Brussels...",today’s meeting held brussels governing counci...
2847,3 April 2014,At today’s meeting the Governing Council of th...,today’s meeting governing council ecb decided ...
2848,6 March 2014,At today’s meeting the Governing Council of th...,today’s meeting governing council ecb decided ...
2849,6 February 2014,At today’s meeting the Governing Council of th...,today’s meeting governing council ecb decided ...


In [61]:
import pandas as pd

def consolidate_lines(df):
    """
    Consolidate lines in the content of a dataframe.
    If a line does not end with a full stop, it is merged
    with the next line, removing unnecessary newline characters.

    Args:
        df (pandas.DataFrame): Dataframe with 'content' column

    Returns:
        pandas.DataFrame: Modified dataframe with consolidated lines
    """
    consolidated_data = []

    for index, row in df.iterrows():
        content = row['content']
        lines = content.split('\n')
        consolidated_content = ""
        for i, line in enumerate(lines):
            line = line.strip()
            if line and (line[-1] in ".!?" or i == len(lines) - 1):
                consolidated_content += line + " "  # Append a space after each line
            else:
                consolidated_content += line + " "  # Append a space to separate lines

        consolidated_row = row.copy()
        consolidated_row['content'] = consolidated_content.strip()
        consolidated_data.append(consolidated_row)

    # Create new dataframe with consolidated content
    new_df = pd.DataFrame(consolidated_data)

    return new_df



import re

def split_paragraphs(df):
    """
    Splits content of dataframe into separate rows. 
    If a paragraph exceeds 512 words, it is limited to 
    two nearly equal parts, with the split done at a full stop.
    
    Args:
        df (pandas.DataFrame): Dataframe with 'date' and 'content' columns
    
    Returns:
        pandas.DataFrame: Modified dataframe with split content
    """
    data = []

    for index, row in df.iterrows():
        date = row['date']
        content = row['content']

        # Split content into sentences
        sentences = re.split(r'(?<=[.!?])\s+', content)

        # Process each sentence
        current_part = ""
        for sentence in sentences:
            if len(current_part.split()) + len(sentence.split()) <= 200:
                current_part += " " + sentence
            else:
                if current_part.strip():
                    data.append([date, current_part.strip()])
                current_part = sentence

        # Append remaining part if exists
        if current_part.strip():
            data.append([date, current_part.strip()])

    # Create new dataframe
    new_df = pd.DataFrame(data, columns=['date', 'content'])

    return new_df

# clean the content column and remove stopwords
import string
import nltk
from nltk.corpus import stopwords

# Download stopwords if you haven't already
nltk.download('stopwords')

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Replace newline characters with space
    text = text.replace('\n', ' ')
    
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Strip extra whitespaces
    text = ' '.join(text.split())
    
    return text

all_data = consolidate_lines(all_data)
all_data = split_paragraphs(all_data)
all_data['content_clean'] = all_data['content'].apply(clean_text)
all_data

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ozodbek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,date,content,content_clean
0,1998-07-17,"Mr. Duisenberg reports on the outcome of the second meeting of the Governing Council of the European Central Bank Introductory statement by the President of the European Central Bank, Dr. W. Duisenberg, at the press conference held in Frankfurt on 8/7/98. The Governing Council first assessed current economic developments in the euro area. The general picture is one of continued economic expansion combined with broadly low inflation. Several forecasts made during spring 1998 have even suggested slightly lower rates of inflation for 1998-99 compared with expectations in autumn 1997. As far as price developments are concerned, inflation as measured by the harmonised consumer price index (HICP) is estimated to have risen slightly in April, to 1.4%, but has not increased further in May. Output growth has remained strong in recent quarters, with annual growth rates close to or even exceeding 3%. Economic growth has been driven increasingly by domestic demand rather than net exports, as might be expected at this stage of the cycle. Private consumption and stockbuilding have been the main factors underlying domestic demand to date.",mr duisenberg reports outcome second meeting governing council european central bank introductory statement president european central bank dr w duisenberg press conference held frankfurt 8798 governing council first assessed current economic developments euro area general picture one continued economic expansion combined broadly low inflation several forecasts made spring 1998 even suggested slightly lower rates inflation 199899 compared expectations autumn 1997 far price developments concerned inflation measured harmonised consumer price index hicp estimated risen slightly april 14 increased may output growth remained strong recent quarters annual growth rates close even exceeding 3 economic growth driven increasingly domestic demand rather net exports might expected stage cycle private consumption stockbuilding main factors underlying domestic demand date
1,1998-07-17,"The favourable conjunctural situation has started to feed through into the labour market, although improvements here have in some countries been too slow to appear. It is evident, however, that economic growth alone will not reduce the urgency to proceed with structural adjustments in order to tackle unacceptably high rates of unemployment. As regards monetary and financial developments in the euro area, the annual growth rate of the broad money supply (M3H) accelerated in April to around 6% from 5.5% in March. The growth rate of M1, the narrow money supply, also picked up, to around 10%. We shall monitor these developments very carefully. In principle, the economic performance I have just described provides a favourable environment for continued fiscal consolidation. It is with some concern that we observe that in a number of member countries the prospects for this to actually develop seem to be, to put it mildly, rather subdued. In this respect, I should like to underline three aspects. First, any “growth dividend” resulting from the current cyclical upturn should be used to bring deficit and debt levels down, rather than relaxing control over expenditure.",favourable conjunctural situation started feed labour market although improvements countries slow appear evident however economic growth alone reduce urgency proceed structural adjustments order tackle unacceptably high rates unemployment regards monetary financial developments euro area annual growth rate broad money supply m3h accelerated april around 6 55 march growth rate m1 narrow money supply also picked around 10 shall monitor developments carefully principle economic performance described provides favourable environment continued fiscal consolidation concern observe number member countries prospects actually develop seem put mildly rather subdued respect like underline three aspects first “growth dividend” resulting current cyclical u

In [16]:
all_data.to_parquet('src/Output/all_data.parquet')


In [62]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX130


In [63]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [64]:
import regex
# eliminate non-tokenizable characters

def isEnglish(text):
    # This pattern matches characters that belong to various common scripts, including Latin, Greek, Cyrillic, etc.
    # It also matches common punctuation, numbers, and spaces.
    pattern = r'^[\p{Script=Latin}\p{Script=Greek}\p{Script=Cyrillic}\p{Number}\p{Punctuation}\p{Zs}]+$'

    return bool(regex.match(pattern, text))

text = 'axa'
isEnglish(text)


True

In [65]:
all_data['is_english'] = all_data['content_clean'].apply(isEnglish)
all_data

,date,content,content_clean,is_english
0,1998-07-17,"Mr. Duisenberg reports on the outcome of the second meeting of the Governing Council of the European Central Bank Introductory statement by the President of the European Central Bank, Dr. W. Duisenberg, at the press conference held in Frankfurt on 8/7/98. The Governing Council first assessed current economic developments in the euro area. The general picture is one of continued economic expansion combined with broadly low inflation. Several forecasts made during spring 1998 have even suggested slightly lower rates of inflation for 1998-99 compared with expectations in autumn 1997. As far as price developments are concerned, inflation as measured by the harmonised consumer price index (HICP) is estimated to have risen slightly in April, to 1.4%, but has not increased further in May. Output growth has remained strong in recent quarters, with annual growth rates close to or even exceeding 3%. Economic growth has been driven increasingly by domestic demand rather than net exports, as might be expected at this stage of the cycle. Private consumption and stockbuilding have been the main factors underlying domestic demand to date.",mr duisenberg reports outcome second meeting governing council european central bank introductory statement president european central bank dr w duisenberg press conference held frankfurt 8798 governing council first assessed current economic developments euro area general picture one continued economic expansion combined broadly low inflation several forecasts made spring 1998 even suggested slightly lower rates inflation 199899 compared expectations autumn 1997 far price developments concerned inflation measured harmonised consumer price index hicp estimated risen slightly april 14 increased may output growth remained strong recent quarters annual growth rates close even exceeding 3 economic growth driven increasingly domestic demand rather net exports might expected stage cycle private consumption stockbuilding main factors underlying domestic demand date,True
1,1998-07-17,"The favourable conjunctural situation has started to feed through into the labour market, although improvements here have in some countries been too slow to appear. It is evident, however, that economic growth alone will not reduce the urgency to proceed with structural adjustments in order to tackle unacceptably high rates of unemployment. As regards monetary and financial developments in the euro area, the annual growth rate of the broad money supply (M3H) accelerated in April to around 6% from 5.5% in March. The growth rate of M1, the narrow money supply, also picked up, to around 10%. We shall monitor these developments very carefully. In principle, the economic performance I have just described provides a favourable environment for continued fiscal consolidation. It is with some concern that we observe that in a number of member countries the prospects for this to actually develop seem to be, to put it mildly, rather subdued. In this respect, I should like to underline three aspects. First, any “growth dividend” resulting from the current cyclical upturn should be used to bring deficit and debt levels down, rather than relaxing control over expenditure.",favourable conjunctural situation started feed labour market although improvements countries slow appear evident however economic growth alone reduce urgency proceed structural adjustments order tackle unacceptably high rates unemployment regards monetary financial developments euro area annual growth rate broad money supply m3h accelerated april around 6 55 march growth rate m1 narrow money supply also picked around 10 shall monitor developments carefully principle economic performance described provides favourable environment continued fiscal consolidation concern observe number member countries prospects actually develop seem put mildly rather subdued respect like underline three aspects first “growth dividend” resulting cu

In [66]:
all_data = all_data[all_data['is_english']==True]
all_data_sample = all_data.sample(n=100, random_state=1)
all_data_sample

,date,content,content_clean,is_english
23048,2022-02-23,"So, we have already come a long way and our supervision of C&E risks will now take more concrete form. The 2022 SREP cycle will be marked by several key initiatives, which represent the next level of maturity for our supervision of C&E risks. The first of these initiatives was the climate risk stress test, launched in January and currently being conducted by a dedicated supervisory team. This stress test was designed, first and foremost, as a learning exercise – for supervisors, for banks, and for the financial industry as whole. We are adamant about the importance of raising awareness of C&E risks. And this stress test constitutes an unprecedented effort, also on our part, to understand more fully how far the 2/4 BIS central bankers' speeches banks we supervise are exposed to these risks. It will also give us a clearer picture of their resilience on this front. The second of these initiatives is a comprehensive review of banks’ practices related to strategy, governance and risk management – the thematic review on C&E risks, which is a key topic of the session today.",already come long way supervision ce risks take concrete form 2022 srep cycle marked several key initiatives represent next level maturity supervision ce risks first initiatives climate risk stress test launched january currently conducted dedicated supervisory team stress test designed first foremost learning exercise – supervisors banks financial industry whole adamant importance raising awareness ce risks stress test constitutes unprecedented effort also part understand fully far 24 bis central bankers speeches banks supervise exposed risks also give us clearer picture resilience front second initiatives comprehensive review banks’ practices related strategy governance risk management – thematic review ce risks key topic session today,True
14207,2013-05-24,"In some euro area countries, this phenomenon is driving youth unemployment to levels that threaten the very fabric of society. Since the crisis started, there has been important progress in labour market reforms and fiscal consolidation in euro area countries, particularly those under an EU/IMF programme. And the painful measures taken are starting to bear fruit. We see this very clearly, for instance, in the impressive improvement in export performance in Ireland, Spain and Portugal and in the recent uptick in industrial production in the latter two countries. The role of European policies and progress towards a banking union Let me turn now to my third and final theme. National policies alone are not enough to leave the crisis behind us once and for all. Policies at the European level are also necessary. For one, European measures to complete the single market, especially in the area of services, are needed. And I know that this is a need that is urgently felt in this country. However, major progress has been made in defining a vision and a path towards a genuine economic and monetary union. The banking union is a major element of this effort.",euro area countries phenomenon driving youth unemployment levels threaten fabric society since crisis started important progress labour market reforms fiscal consolidation euro area countries particularly euimf programme painful measures taken starting bear fruit see clearly instance impressive improvement export performance ireland spain portugal recent uptick industrial production latter two countries role european policies progress towards banking union let turn third final theme national policies alone enough leave crisis behind us policies european level also necessary one european measures complete single market especially area services needed know need urgently felt country however major progress made defining vision path towards genuine economic monetary union banking union major element effort,True
23528,23 September 1999,"This adaptation, which is consistent with the overall set-up, is considered us

In [67]:
import torch
print(torch.cuda.is_available())


True


In [68]:
from transformers import BertForSequenceClassification, BertTokenizer, pipeline

# Use a GPU device if available
device = 0 if torch.cuda.is_available() else -1

model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

nlp_classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [69]:
import pandas as pd
all_data_sample['content'] = all_data_sample['content'].astype(str)
all_data_sample = all_data_sample.dropna(subset=['content'])
all_data_sample

,date,content,content_clean,is_english
23048,2022-02-23,"So, we have already come a long way and our supervision of C&E risks will now take more concrete form. The 2022 SREP cycle will be marked by several key initiatives, which represent the next level of maturity for our supervision of C&E risks. The first of these initiatives was the climate risk stress test, launched in January and currently being conducted by a dedicated supervisory team. This stress test was designed, first and foremost, as a learning exercise – for supervisors, for banks, and for the financial industry as whole. We are adamant about the importance of raising awareness of C&E risks. And this stress test constitutes an unprecedented effort, also on our part, to understand more fully how far the 2/4 BIS central bankers' speeches banks we supervise are exposed to these risks. It will also give us a clearer picture of their resilience on this front. The second of these initiatives is a comprehensive review of banks’ practices related to strategy, governance and risk management – the thematic review on C&E risks, which is a key topic of the session today.",already come long way supervision ce risks take concrete form 2022 srep cycle marked several key initiatives represent next level maturity supervision ce risks first initiatives climate risk stress test launched january currently conducted dedicated supervisory team stress test designed first foremost learning exercise – supervisors banks financial industry whole adamant importance raising awareness ce risks stress test constitutes unprecedented effort also part understand fully far 24 bis central bankers speeches banks supervise exposed risks also give us clearer picture resilience front second initiatives comprehensive review banks’ practices related strategy governance risk management – thematic review ce risks key topic session today,True
14207,2013-05-24,"In some euro area countries, this phenomenon is driving youth unemployment to levels that threaten the very fabric of society. Since the crisis started, there has been important progress in labour market reforms and fiscal consolidation in euro area countries, particularly those under an EU/IMF programme. And the painful measures taken are starting to bear fruit. We see this very clearly, for instance, in the impressive improvement in export performance in Ireland, Spain and Portugal and in the recent uptick in industrial production in the latter two countries. The role of European policies and progress towards a banking union Let me turn now to my third and final theme. National policies alone are not enough to leave the crisis behind us once and for all. Policies at the European level are also necessary. For one, European measures to complete the single market, especially in the area of services, are needed. And I know that this is a need that is urgently felt in this country. However, major progress has been made in defining a vision and a path towards a genuine economic and monetary union. The banking union is a major element of this effort.",euro area countries phenomenon driving youth unemployment levels threaten fabric society since crisis started important progress labour market reforms fiscal consolidation euro area countries particularly euimf programme painful measures taken starting bear fruit see clearly instance impressive improvement export performance ireland spain portugal recent uptick industrial production latter two countries role european policies progress towards banking union let turn third final theme national policies alone enough leave crisis behind us policies european level also necessary one european measures complete single market especially area services needed know need urgently felt country however major progress made defining vision path towards genuine economic monetary union banking union major element effort,True
23528,23 September 1999,"This adaptation, which is consistent with the overall set-up, is considered us

In [70]:
# Convert into list
content_list = all_data_sample['content_clean'].tolist()

# Define the candidate labels
candidate_labels = ["Negative outlook for the economy",
                    "Positive outlook for the economy",
                    "General speech about the economy and no clear sentiment or message",
                    ]

# Perform zero-shot classification
results = nlp_classifier(content_list, candidate_labels)

# Extract the predicted labels
predicted_labels = [result['labels'][0] for result in results]

# Add the predicted labels to the DataFrame
all_data_sample['predicted_label'] = predicted_labels


/home/ozodbek/anaconda3/envs/master_thesis_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7fd11bd8dc40> was reported to be 100 (when accessing len(dataloader)), but 101 samples have been fetched. 
  warnings.warn(warn_msg)
/home/ozodbek/anaconda3/envs/master_thesis_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7fd11bd8dc40> was reported to be 100 (when accessing len(dataloader)), but 102 samples have been fetched. 
  warnings.warn(warn_msg)
/home/ozodbek/anaconda3/envs/master_thesis_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7fd11bd8dc40> was reported to be 100 (when accessing len(dataloader)), but 103 samp

In [71]:
all_data_sample['predicted_label'].value_counts()

predicted_label
Negative outlook for the economy                                      53
Positive outlook for the economy                                      43
General speech about the economy and no clear sentiment or message     4
Name: count, dtype: int64

In [59]:
sent_dic = {
    "Positive outlook for the economy":"Positive",
    "Negative outlook for the economy":"Negative"
}
all_data_sample['sentiment_0shot_binary'] = all_data_sample['predicted_label'].map(sent_dic)

In [60]:
pd.set_option('display.max_colwidth', None)
all_data_sample[all_data_sample["sentiment_0shot_binary"]=="Negative"]

date  \
1027     2012-05-09   
694      2009-09-16   
670      2009-06-18   
991      2011-12-06   
1988     2019-04-08   
...             ...   
2627  3 August 2000   
2263    6 June 2023   
2333   22 July 2022   
779      2010-05-03   
433      2007-10-24   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          